In [73]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from stocksent import Sentiment
import torch
import torch.utils.data as Data
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import yahoo_fin.stock_info as si

In [ ]:
snp500_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/snp500'))
print(len(snp500_files))
nasdaq_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/nasdaq')) - snp500_files
print(len(nasdaq_files))
nyse_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/nyse')) - snp500_files
print(len(nyse_files))
nasdaq_nyse: set = nasdaq_files.union(nyse_files)
print(len(nasdaq_nyse))


In [ ]:
initial_stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/snp500/AMZN.csv')
initial_stock_data['Date'] = initial_stock_data['Date'].apply(lambda x: x[-4:])
annual_data = initial_stock_data.groupby('Date').mean()
annual_data['Annual Percent Change'] = annual_data['Close'].pct_change()
annual_data = annual_data.fillna(0)
print(annual_data.head(20))

In [148]:
news_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/analyst_ratings_processed.csv', index_col=0)
news_data['date'] = news_data['date'].apply(lambda x: str(x)[:4])
print(news_data.head(30))

                                                  title  date stock
0.0             Stocks That Hit 52-Week Highs On Friday  2020     A
1.0          Stocks That Hit 52-Week Highs On Wednesday  2020     A
2.0                       71 Biggest Movers From Friday  2020     A
3.0        46 Stocks Moving In Friday's Mid-Day Session  2020     A
4.0   B of A Securities Maintains Neutral on Agilent...  2020     A
5.0   CFRA Maintains Hold on Agilent Technologies, L...  2020     A
6.0   UBS Maintains Neutral on Agilent Technologies,...  2020     A
7.0   Agilent Technologies shares are trading higher...  2020     A
8.0   Wells Fargo Maintains Overweight on Agilent Te...  2020     A
9.0          10 Biggest Price Target Changes For Friday  2020     A
10.0    30 Stocks Moving in Friday's Pre-Market Session  2020     A
11.0  SVB Leerink Maintains Outperform on Agilent Te...  2020     A
12.0  8 Stocks Moving In Thursday's After-Hours Session  2020     A
13.0  Agilent Technologies shares are trading hi

In [51]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)

def get_sentiment(input_text: str, model=model, tokenizer=tokenizer, device=device):

    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits.to(device)

    return torch.nn.Softmax(dim=1)(logits)[0].tolist()

In [159]:
news_by_stock = news_data.groupby('stock')
already_processed = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/Old/stock_data'))
for stock in tqdm(news_by_stock):
    # if stock[0] + '.csv' in already_processed:
    #     continue
    try:
        stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/snp500/' + stock[0] + '.csv')
        label = 1
    except FileNotFoundError:
        try:
            stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/nasdaq/' + stock[0] + '.csv')
            label = 0
        except FileNotFoundError:
            try:
                stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/nyse/' + stock[0] + '.csv')
                label = 0
            except FileNotFoundError:
                continue
    stock_data['Date'] = stock_data['Date'].apply(lambda x: x[-4:])
    
    annual_data = stock_data.groupby('Date')
    placeholder = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    for year in annual_data:
        try:
            year[1]['Annual Percent Change'] = (year[1].iloc[-1]['close'] - year[1].iloc[0]['close']) / year[1].iloc[0]['close'] * 100
        except:
            print(year[1])
        means = year[1].mean()
        means = means.fillna(0)
        placeholder = pd.concat([placeholder, means], ignore_index=True)


    news_by_year = stock[1].groupby('date')
    sentiments_frame = pd.DataFrame(columns=['positive', 'negative', 'neutral'])
    sentiment_index = [year[0] for year in news_by_year]
    for year in news_by_year:
        stories = [year[1]['title'].iloc[:5].tolist()][0]

        sentiments = pd.DataFrame(columns=['positive', 'negative', 'neutral'], index=[year[0]])
        for story in stories:
            sentiment = get_sentiment(story)
            sentiment_frame = pd.DataFrame({'positive': sentiment[0], 'negative': sentiment[1], 
                               'neutral': sentiment[2]}, index=[year[0]])
            sentiments = pd.concat([sentiments, sentiment_frame], ignore_index=True)
        
        mean_sentiments = sentiments.mean()
        sentiments_frame = pd.concat([sentiments_frame, mean_sentiments], ignore_index=True)
    sentiments_frame.index = sentiment_index
    combined_data = pd.concat([placeholder, sentiments_frame], axis=1)
    combined_data = combined_data.fillna(combined_data.mean())
    combined_data['Label'] = label
    combined_data = combined_data.reset_index(drop=True)
    combined_data.drop(combined_data[combined_data.Volume == 0].index, inplace=True)
    print(combined_data.head(20))
    # combined_data.drop(columns=['Date'], inplace=True)
    # combined_data.to_csv(f'G:/My Drive/UdS/Classes/Data Science/stock_data/{stock[0]}.csv', index=False)
    break

  0%|          | 0/6192 [00:00<?, ?it/s]

    Date        Low       Open    Volume       High      Close  Adjusted Close
0   1999  28.612303  32.546494  62546380  35.765381  31.473534       26.929760
1   1999  28.478184  30.713518  15234146  30.758226  28.880545       24.711119
2   1999  28.657009  29.551144   6577870  31.473534  31.473534       26.929760
3   1999  28.612303  30.400572   5975611  31.205294  28.612303       24.481602
4   1999  28.612303  28.701717   4843231  29.998213  29.372318       25.131901
5   1999  29.148785  29.238197   1729466  29.685265  29.461731       25.208408
6   1999  29.014664  29.327612   4074751  30.355865  30.132332       25.782187
7   1999  29.282904  30.042917   4310034  30.713518  30.177038       25.820436
8   1999  29.953505  30.177038   2957329  31.071173  30.713518       26.279463
9   1999  30.892345  31.294706   3069868  32.188843  31.562946       27.006264
10  1999  31.697067  32.144135   3041629  32.680614  31.831186       27.235783
11  1999  32.322960  32.367668   2250780  33.217094 

  0%|          | 0/6192 [00:03<?, ?it/s]


ValueError: Length mismatch: Expected axis has 36 elements, new values have 12 elements

### Relabel stocks by APR

In [141]:
#relabelled_stocks = os.listdir('G:/My Drive/UdS/Classes/Data Science/relabelled_data')
good_years = 0
for stock in tqdm(os.listdir('G:/My Drive/UdS/Classes/Data Science/Old/truncated_data')):
    if stock == "desktop.ini":
        continue
    # if stock in relabelled_stocks:
    #     continue
    try:
        truncated_data = pd.read_csv(f'G:/My Drive/UdS/Classes/Data Science/Old/truncated_data/{stock}')
    except:
        continue
    for year in truncated_data.index.to_list():
        apr = truncated_data.loc[year, 'Annual Percent Change']
        if apr > 3.8:
            truncated_data.loc[year, 'Label'] = 2
            good_years += 1
        elif apr > 0:
            truncated_data.loc[year, 'Label'] = 1
        else:
            truncated_data.loc[year, 'Label'] = 0
    truncated_data.to_csv(f'G:/My Drive/UdS/Classes/Data Science/relabelled_data/{stock}', index=False)

100%|██████████| 2337/2337 [01:54<00:00, 20.47it/s]


In [ ]:
prepared_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/stock_data')
#processed_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/truncated_data')

recent_year: dict = {}
for stock in tqdm(prepared_stock_list):
    if stock == 'desktop.ini':
        continue
    # if stock in processed_stock_list:
    #     continue
    stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/stock_data/' + stock)
    try:
        recent_year[stock[:-4]] = stock_data.iloc[-1]
    except IndexError:
        print(stock)
        print(stock_data)
        print(stock_data.head())
        continue
    # write to csv, truncating first and last line
    #stock_data[1:-1].to_csv('G:/My Drive/UdS/Classes/Data Science/truncated_data/' + stock, index=False)
    
recent_year_dataframe = pd.DataFrame.from_dict(recent_year, orient='index')

# recent_year_dataframe.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/toy_data/stonks.csv', index=True)

In [142]:
relabelled_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/relabelled_data')
#relabelled_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/truncated_data')

columns = ["Low","Open","Volume","High","Close","Adjusted Close","Annual Percent Change","positive","negative","neutral","Label"]
full_dataset = pd.DataFrame(columns=columns)
for stock in tqdm(relabelled_stock_list):
    if stock == 'desktop.ini':
        continue
    try:
        stock_data = pd.read_csv(f'G:/My Drive/UdS/Classes/Data Science/relabelled_data/{stock}')
    except:
        continue
    try:
        full_dataset = pd.concat([full_dataset, stock_data], ignore_index=True)
    except IndexError:
        print(stock)
        print(stock_data)
        print(stock_data.head())
        continue
# write to csv
full_dataset.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/stonks.csv', index=False)


100%|██████████| 2337/2337 [00:51<00:00, 44.97it/s]


In [143]:
stonks = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/stonks.csv')
groups = stonks.groupby('Label')
print(groups.size())

Label
0    21606
1    30970
2       76
dtype: int64


In [144]:
for value in stonks['Annual Percent Change']:
    if value > 6.829:
        print(value)

9.252720896233214
7.374908936514991
27.944679108181216
7.111712728278983
10.167301188480309
7.584222405623883
25.25999234517084
9.55686637978917
7.096756805318481
16.054298604249507
13.014807329863068
7.413227101242988
8.04909488441505
8.312242660700823
11.553498066019475
8.135831980458489
25.84447076858756
320.7978593920884
15.760884392651144
11.565520176850072
7.872404729238704
7.011025937204662
7.727659259420024
6.858298962629672
17.53198494159436
7.625809065699372
9.134205607582736
109540.9028466044
9.504233217086188


In [ ]:
y = full_dataset["Label"]
X = full_dataset.drop(columns=["Label"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80:20 train to test
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.5, random_state=1) # 50:50 test to dev > 80:10:10 train:dev:test

# write splits to individual csv files
# X_train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_train.csv', index=False)
# X_dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_dev.csv', index=False)
# X_test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_test.csv', index=False)
# y_train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_train.csv', index=False)
# y_dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_dev.csv', index=False)
# y_test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_test.csv', index=False)

### Reload data for training

In [2]:
# reload splits from csv files
X_train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_train.csv')
X_dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_dev.csv')
X_test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/X_test.csv')
y_train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_train.csv')
y_dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_dev.csv')
y_test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/y_test.csv')

# verify data integrity
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)
print(y_train.shape)
print(y_dev.shape)
print(y_test.shape)


(45859, 10)
(5733, 10)
(5732, 10)
(45859, 1)
(5733, 1)
(5732, 1)


In [62]:
# train = pd.concat([X_train, y_train], axis=1)
# dev = pd.concat([X_dev, y_dev], axis=1)
# test = pd.concat([X_test, y_test], axis=1)

# write splits to individual csv files
# train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/train.csv', index=False)
# dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/dev.csv', index=False)
# test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/test.csv', index=False)

# reload splits from csv files
train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/train.csv')
dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/dev.csv')
test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/test.csv')


In [63]:
print(train.shape)
print(dev.shape)
print(test.shape)

(45859, 11)
(5733, 11)
(5732, 11)


In [64]:
train_dataset = train.copy()
y_train_indices = train_dataset.index

y_train = [train_dataset.Label[i] for i in y_train_indices]
#y_train = train['Label']

class_sample_count = np.array(
    [len(np.where(y_train == t)[0]) for t in np.unique(y_train)])
print(class_sample_count)
print(y_train)
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)
sampler = Data.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_dataset.drop(columns=["Label"], inplace=True)
print(train_dataset.head())
train_dataset = Data.TensorDataset(torch.tensor(train_dataset.values), torch.tensor(y_train))
print(train_dataset[0])

[39783  6076]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [148]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train_X = torch.tensor(X_train.values, dtype=torch.float32)
# train_y = torch.tensor(y_train.values, dtype=torch.long)

class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

# traindata = MyDataset(train_X, train_y)
# trainloader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)

train_dataloader = Data.DataLoader(train_dataset, batch_size=64, sampler=sampler, num_workers=0)


model = torch.nn.Sequential(
        torch.nn.Linear(10, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1),
        torch.nn.Softmax()
)

model.train()

#loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
ep_log_interval = 100
loss_fn = torch.nn.MSELoss()
num_epochs = 100


for n in range(num_epochs):
    epoch_loss = 0.0
    for (batch_idx, batch) in enumerate(train_dataloader):
        X = batch[0].to(torch.float32)  # inputs
        Y = batch[1].to(torch.float32)  # correct class/label/politics
        optimizer.zero_grad()
        oupt = model(X).squeeze(1).to(torch.float32)  # 1D tensor for each sample
        loss_val = loss_fn(oupt, Y)  # a tensor
        epoch_loss += loss_val.item()  # accumulate
        loss_val.backward()
        optimizer.step()
    print("epoch = %d, loss = %f" % (n, epoch_loss))

# for n in tqdm(range(num_epochs)):
#     for X, y in train_dataloader:
#         X = X.to(torch.float32)
#         y = y
#         y_pred = model(X).squeeze(1)
#         loss = loss_fn(y_pred, y).item()
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

print(model)

torch.save(model, "G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/models/multilayer_model.pickle")

c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch = 0, loss = 360.509375
epoch = 1, loss = 358.152232
epoch = 2, loss = 357.876339
epoch = 3, loss = 361.764286
epoch = 4, loss = 359.485714
epoch = 5, loss = 356.331696
epoch = 6, loss = 358.493304
epoch = 7, loss = 359.175446
epoch = 8, loss = 356.300893
epoch = 9, loss = 358.951786
epoch = 10, loss = 358.709821
epoch = 11, loss = 360.141518
epoch = 12, loss = 355.360714
epoch = 13, loss = 358.074107
epoch = 14, loss = 356.540179
epoch = 15, loss = 360.396875
epoch = 16, loss = 360.160268
epoch = 17, loss = 360.133929
epoch = 18, loss = 355.209821
epoch = 19, loss = 357.629018
epoch = 20, loss = 358.366071


KeyboardInterrupt: 

In [117]:
# dev_X = torch.tensor(X_dev.values, dtype=torch.float32)
# dev_y = torch.tensor(y_dev.values, dtype=torch.long)

dev_data = dev.copy()

y_dev = [dev_data.Label[i] for i in dev_data.index]
dev_data.drop(columns=["Label"], inplace=True)
dev_dataset = Data.TensorDataset(torch.tensor(dev_data.values), torch.tensor(y_dev))

class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]


dev_loader = Data.DataLoader(dev_dataset, batch_size=64)

model = torch.load("G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/models/multilayer_model.pickle")
print(model)
model.eval()

dev_predictions = pd.DataFrame(columns=['y_pred', 'y_true'])
with torch.no_grad():
    for X, y in dev_loader:
        X = X.to(torch.float32)
        y = y
        y_pred = model(X).squeeze(1)
        y_frame = pd.DataFrame({'y_pred': y_pred, 'y_true': y})
        dev_predictions = dev_predictions.append(y_frame, ignore_index=True)

confusion = pd.crosstab(dev_predictions['y_true'], dev_predictions['y_pred'], rownames=['True'], 
                        colnames=['Predicted'], margins=True)
print(confusion)

try:
    accuracy = (confusion[0][0] + confusion[1][1]) / confusion['All']['All']
except KeyError:
    accuracy = (confusion[0][0]) / confusion['All']['All']
print("accuracy:", accuracy)
try:
    precision = confusion[1][1] / confusion['All'][1]
except KeyError:
    precision = 0
print("precision:", precision)
try:
    recall = confusion[1][1] / confusion[1]['All']
except KeyError:
    recall = 0
print("recall:", recall)
try:
    f1 = 2 * (precision * recall) / (precision + recall)
except ZeroDivisionError:
    f1 = 0
print("f1:", f1)

Sequential(
  (0): Linear(in_features=10, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
  (5): LogSoftmax(dim=1)
)
Predicted   0.0   All
True                 
0          4970  4970
1           763   763
All        5733  5733
accuracy: 0.8669108669108669
precision: 0
recall: 0
f1: 0


In [113]:
def get_stock_data(ticker: str) -> list:
    today: datetime = datetime.today()
    yesteryear: str = (today - timedelta(days=345)).strftime('%Y-%m-%d')
    column_names: "list[str]" = ["low","open","volume","high","close","adjclose","Annual Percent Change","positive","negative","neutral"]

    stock_news = Sentiment(ticker)
    sentiment_score = stock_news.get_dataframe(days=1)
    stories = sentiment_score['headline'].tolist()[:10]
    sentiments: "list[float]" = []

    for story in stories:
        sentiment = get_sentiment(story)
        sentiments.append(sentiment)

    sentiments_df = pd.DataFrame(sentiments, columns=['positive', 'negative', 'neutral'])
    mean_sentiments: pd.Series = sentiments_df.mean()
    stock_sentiment: "list[float]" = mean_sentiments.values.tolist()

    ticker_info: pd.DataFrame = si.get_data(ticker, start_date=yesteryear, end_date=today, interval="1mo")
    ticker_info = ticker_info.drop(columns=['ticker'])
    ticker_info['Annual Percent Change'] = (ticker_info.iloc[-1]['close'] - ticker_info.iloc[0]['close']) / ticker_info.iloc[0]['close'] * 100
    print(ticker_info['Annual Percent Change'])
    annualized_ticker_info: pd.Series = ticker_info.mean()

    annualized_ticker_info['positive'] = stock_sentiment[0]
    annualized_ticker_info['negative'] = stock_sentiment[1]
    annualized_ticker_info['neutral'] = stock_sentiment[2]

    annualized_ticker_info = annualized_ticker_info[column_names] # reorder columns

    return annualized_ticker_info.values.tolist()


In [114]:
print(get_stock_data("AAPL"))

c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\stocksent\get_sentiment_data.py:97: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  news_df['date'] = pd.to_datetime(news_df.date).dt.date


2022-08-01    18.164733
2022-09-01    18.164733
2022-10-01    18.164733
2022-11-01    18.164733
2022-12-01    18.164733
2023-01-01    18.164733
2023-02-01    18.164733
2023-03-01    18.164733
2023-04-01    18.164733
2023-05-01    18.164733
2023-06-01    18.164733
2023-06-22    18.164733
Name: Annual Percent Change, dtype: float64
[148.65333239237467, 156.26833470662436, 1359916645.5, 166.4008331298828, 158.33238093058267, 157.92682011922201, 18.164733194417078, 0.11016231784597039, 0.16802884330973028, 0.7218088336288929]
